In [1]:
import cv2
from deepface import DeepFace
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import numpy as np
import threading

In [2]:
import cv2
from deepface import DeepFace
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import numpy as np
import threading

class MoodDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Mood Detection System")
        self.root.geometry("800x600")

        # Video variables
        self.cap = cv2.VideoCapture(0)
        self.is_running = False

        # UI Elements
        self.video_label = ttk.Label(root)
        self.video_label.pack(pady=10)

        self.mood_label = ttk.Label(root, text="Mood: Detecting...", font=("Arial", 14))
        self.mood_label.pack(pady=10)

        self.confidence_label = ttk.Label(root, text="Confidence: 0%", font=("Arial", 12))
        self.confidence_label.pack(pady=5)

        # Buttons
        self.start_button = ttk.Button(root, text="Start Detection", command=self.start_detection)
        self.start_button.pack(pady=5)

        self.stop_button = ttk.Button(root, text="Stop Detection", command=self.stop_detection)
        self.stop_button.pack(pady=5)
        self.stop_button.config(state="disabled")

        self.quit_button = ttk.Button(root, text="Quit", command=self.quit_app)
        self.quit_button.pack(pady=5)

    def start_detection(self):
        if not self.is_running:
            self.is_running = True
            self.start_button.config(state="disabled")
            self.stop_button.config(state="normal")
            threading.Thread(target=self.detect_mood, daemon=True).start()

    def stop_detection(self):
        self.is_running = False
        self.start_button.config(state="normal")
        self.stop_button.config(state="disabled")
        self.mood_label.config(text="Mood: Detecting...")
        self.confidence_label.config(text="Confidence: 0%")

    def detect_mood(self):
        while self.is_running:
            ret, frame = self.cap.read()
            if not ret:
                break

            try:
                # Analyze mood using DeepFace
                result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
                
                # Get dominant emotion and confidence
                dominant_emotion = result[0]['dominant_emotion']
                confidence = result[0]['emotion'][dominant_emotion]

                # Update UI
                self.mood_label.config(text=f"Mood: {dominant_emotion.capitalize()}")
                self.confidence_label.config(text=f"Confidence: {confidence:.2f}%")

                # Draw rectangle around face and add mood text
                face_location = result[0]['region']
                x, y, w, h = face_location['x'], face_location['y'], face_location['w'], face_location['h']
                
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, f"{dominant_emotion} ({confidence:.2f}%)", 
                           (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            except Exception as e:
                self.mood_label.config(text="Mood: Face not detected")
                self.confidence_label.config(text="Confidence: 0%")

            # Convert frame for Tkinter
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame_rgb)
            imgtk = ImageTk.PhotoImage(image=img)
            
            # Update video feed
            self.video_label.configure(image=imgtk)
            self.video_label.image = imgtk

    def quit_app(self):
        self.is_running = False
        self.cap.release()
        self.root.quit()

def main():
    # Check if webcam is available
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return

    root = tk.Tk()
    app = MoodDetectionApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()